## Graph Similarity Search Project ##
### This is the implementation for the similarity search project using python with Neo4j

In [1]:
# This program fetches the required user's facebook data via the free facebook api.
# It currently fetches friend's data, but it can be extended.
# Author: Ajose Opeyemi

# importing the requests library for fetching the required webpage
import requests as r
import pandas as pd
import json

# setting up parameters
# Importing the "friend's" list downloaded from facebook
file = r"C:\Users\Victor\Desktop\Bae\Code files\facebook-Elvicharde\friends_and_followers\friends.json"
with open(file) as friends_data:
    friends = json.load(friends_data)
    friends_data.close()

In [2]:
# Parsing and handling the data

me = "Elvicharde"
friends_list = friends['friends_v2']

for friend in friends_list:
    #cleaning up the imported data
    friend['ID'] = friend.pop('timestamp')
    friend['Friends'] = friend.pop('name')
    
friends_data = pd.DataFrame(friends_list)    #Setting up a dataframe
del friends_data['contact_info']    # Cleaning up the dataframe.
print(friends_data)    # Printing the first 3 rows

             ID                   Friends
0    1627383250               Collins Obi
1    1626987935  Ademola Ebenezer Adeyemi
2    1626987929           Blessing Mathew
3    1626987884               Tosin Akeem
4    1622810997          Olaniyi Akintayo
..          ...                       ...
783  1268394361              Sodique Alim
784  1268329470       OluBorode Damie Ope
785  1268329416         OluwaTosin Fabrio
786  1268245796                 Sorad Jnr
787  1268242864              Demi Dinakin

[788 rows x 2 columns]


## Interfacing with Neo4j for visualization and query

In [3]:
# Creating the connection
from neo4j import GraphDatabase, basic_auth
from random import randint as randint


####### import data into neo4j ##### 
#from neo4j import GraphDatabase

friends_list = friends_data.values.tolist()
commands = []

# illegal_names = [1603558328,1589458329,1576927897,1574813063,1574452201,
#                  1571760211,1548836078,1504036137,1499675661,1451300849,1304871572]

root_user = 'CREATE (root:user {Name:"Elvicharde", ID:"root_node"})'    # The logged-in user/ account holder

commands.append(root_user)    #Adding the user creation and alias to the cypher query list

# Creating friends and matching to the root node
for i in friends_list:
#    if ((i[0] not in illegal_names) and (i[1].find('-') < 0)):
    create_friends = (f'CREATE (f:friend {{Name: "{i[1]}", ID: {i[0]}}})')
    create_relationships = f'MATCH (f:friend{{Name:"{i[1]}"}}), (root:user{{Name:"Elvicharde"}}) CREATE (f)-[:FRIENDS_WITH{{since: {randint(2010, 2021)}}}]->(root) RETURN * LIMIT 1'
    # else:
#     continue
    commands.append(create_friends)
    commands.append(create_relationships)
commands[:5]

['CREATE (root:user {Name:"Elvicharde", ID:"root_node"})',
 'CREATE (f:friend {Name: "Collins Obi", ID: 1627383250})',
 'MATCH (f:friend{Name:"Collins Obi"}), (root:user{Name:"Elvicharde"}) CREATE (f)-[:FRIENDS_WITH{since: 2020}]->(root) RETURN * LIMIT 1',
 'CREATE (f:friend {Name: "Ademola Ebenezer Adeyemi", ID: 1626987935})',
 'MATCH (f:friend{Name:"Ademola Ebenezer Adeyemi"}), (root:user{Name:"Elvicharde"}) CREATE (f)-[:FRIENDS_WITH{since: 2015}]->(root) RETURN * LIMIT 1']

In [4]:
def execute_commands(commands):
    data_base_connection = GraphDatabase.driver(#uri = #"bolt://54.89.206.251:7687", auth=basic_auth("neo4j", "advancement-characteristic-energy")) #for sandbox connection
    uri = 'bolt://localhost:7687', auth = basic_auth('neo4j','similarity')) # for local dbms connection

    session = data_base_connection.session()

    # re-setting database to clean status
    if session.run('MATCH (n) RETURN n'):
        database_cleanup = 'MATCH (n) DETACH DELETE (n)'
        session.run(database_cleanup)

    # Populating with new data
    for i in commands:
        session.run(i)

        
execute_commands(commands)

## Running The Graph algorithms

In [ ]:
# First algorithm
